In [ ]:
if True:
    import numpy as np
    import pandas as pd

    # Add lab library
    import sys
    sys.path.insert(0, '/home/trevormjs/Documents/Science/APL/Lab')

    #---------------------------------------------------------------------#
    #                        matplotlib plotting                          #
    #---------------------------------------------------------------------#
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    from jupyterthemes import jtplot
    from Helper.plotting import my_graph
    # Edit the font, font size, and axes width
    # mpl.rcParams['font.family'] = 'Avenir'
    plt.rcParams['font.size'] = 24
    plt.rcParams['axes.linewidth'] = 2
    jtplot.style(theme='monokai', context='notebook', ticks=False, grid=False)

    #---------------------------------------------------------------------#
    #                           bokeh plotting                            #
    #---------------------------------------------------------------------#
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.themes import Theme
    from bokeh.io import curdoc, export_png
    from bokeh.models import Range1d, Label, ColumnDataSource, LabelSet
    from Helper.plotting import style
    output_notebook()
    # curdoc().theme = Theme(filename="../Helper/theme.yml")

    #---------------------------------------------------------------------#
    #                       error and unit handling                       #
    #---------------------------------------------------------------------#
    from uncertainties import ufloat
    import Helper.numbers as nu
    from Helper.record import Measurement, Unit

    %load_ext autoreload
    %autoreload 2

In [ ]:
def read_scope_csv(path, n_sigs = 1):
    ret = []
    for n in range(n_sigs):
        config = pd.read_csv(path, header=None, usecols=[1+6*n, 2+6*n]).loc[:2]
        config = [record_length, sample_interval, trigger_point] = [
            [float(val), unit] for val, unit in zip(config[1+6*n], config[2+6*n])]
        config= {
            'record_length': record_length,
            'sample_interval': sample_interval,
            'trigger_point': trigger_point
        }
        display(config)
        data = pd.read_csv(path, header = None, usecols=[3+6*n, 4+6*n])
        data.columns = ['ts', 'mV']
        ret.append([data, config])
    return ret

In [ ]:
def clean_vis_9010_diff(pd1, pd2, first = False, filt_type='fft', filter_param=.25, plot=True):
    fig = figure(height = 400)
    if filt_type == 'fft':
        pd1_clean = pd.Series(nu.fft_filter(pd1.mV, filter_param))
        pd2_clean = pd.Series(nu.fft_filter(pd2.mV, filter_param))
    elif filt_type == 'avg':
        pd1_clean = pd.Series(nu.moving_avg_filter(pd1.mV, filter_param))
        pd2_clean = pd.Series(nu.moving_avg_filter(pd2.mV, filter_param))
    else:
        raise ValueError('No filter match')

    print('pd1 unfiltered ', end = '')
    pd1_start, pd1_end = nu.rising_edge(pd1_clean, pd1.ts, first)[0]
    print('pd2 unfiltered ', end = '')
    pd2_start, pd2_end = nu.rising_edge(pd2_clean, pd2.ts, first)[0]

    # timerange = [pd1_start-25, pd2_end + 150]
    # pd1_clean = pd1_clean[timerange[0]:timerange[1]]
    # pd2_clean = pd2_clean[timerange[0]:timerange[1]]

    t = pd1.ts - pd1.ts[pd1_start]  # [timerange[0]:timerange[1]]
    t_interp = np.linspace(t.min(), t.max(), len(pd2_clean)*16)

    interp1 = interp1d(t, pd1_clean, kind='cubic')
    pd1_clean = pd.Series(interp1(t_interp))

    interp2 = interp1d(t, pd2_clean, kind='cubic')
    pd2_clean = pd.Series(interp2(t_interp))

    t = t_interp

    print('pd1 filtered ', end = '')
    pd1_inds, pd1_90_10_inds = nu.rising_edge(pd1_clean, t, first)
    fig.scatter(t[pd1_inds], pd1_clean[pd1_inds])
    fig.scatter(t[pd1_90_10_inds], pd1_clean[pd1_90_10_inds], color='skyblue')

    print('pd2 filtered ', end = '')
    pd2_inds, pd2_90_10_inds = nu.rising_edge(pd2_clean, t, first)
    fig.scatter(t[pd2_inds], pd2_clean[pd2_inds])
    fig.scatter(t[pd2_90_10_inds], pd2_clean[pd2_90_10_inds], color='skyblue')


    small_t = t[pd1_inds[0]-50:pd1_inds[0] + (pd1_inds[1]-pd1_inds[0])*2+100]
    small_pd1 = pd1_clean[pd1_inds[0]-50:pd1_inds[0] + (pd1_inds[1]-pd1_inds[0])*2+100]
    
    peaks, b, b = nu.peak(small_pd1, 1, 0, 0, 0)
    widths, _, _, _ = nu.peak_widths(small_pd1, peaks,)
    print('pd1 peak halfmax:', widths[0] * (t[1]-t[0]))
    
    
    small_t = t[pd2_inds[0]-50:pd2_inds[0] + (pd2_inds[1]-pd2_inds[0])*2+100]
    small_pd2 = pd2_clean[pd2_inds[0]-50:pd2_inds[0] + (pd2_inds[1]-pd2_inds[0])*2+100]
    
    peaks, b, b = nu.peak(small_pd2, 1, 0, 0, 0)
    widths, _, _, _ = nu.peak_widths(small_pd2, peaks,)
    print('pd2 peak halfmax:', widths[0] * (t[1]-t[0]))
    
    
    fig.line(t[pd1_inds[0]-100:pd2_inds[1]*2], pd1_clean[pd1_inds[0]-100:pd2_inds[1]*2], legend_label='PD1')
    fig.line(t[pd1_inds[0]-100:pd2_inds[1]*2], pd2_clean[pd1_inds[0]-100:pd2_inds[1]*2], legend_label='PD2', color='red')

    fig.line(t[pd1_inds[0]:pd1_inds[1]], pd1_clean[pd1_inds[0]:pd1_inds[1]], color = 'green', alpha = 1)
    fig.line(t[pd2_inds[0]:pd2_inds[1]], pd2_clean[pd2_inds[0]:pd2_inds[1]], color = 'green', alpha = 1)

    
    fig.x_range = Range1d(t[pd1_inds[0]] - 5e-10, t[pd2_inds[1]]*1.3)
    if plot:
        show(fig)
    else:
        del fig

    return pd.DataFrame({
        'start': t[[pd1_inds[0], pd2_inds[0]]],
        'end': t[[pd1_inds[1], pd2_inds[1]]],
        '10': t[[pd1_90_10_inds[0], pd2_90_10_inds[0]]],
        '90': t[[pd1_90_10_inds[1], pd2_90_10_inds[1]]]
    }, index=['sig1', 'sig2'])

## Test Pulse, Scope, and Laser Diode

In [ ]:
[[squares, squares_config]] = read_scope_csv('./Data/l4_A_1_data.csv')

In [ ]:
fig = figure()
fig.line(squares.ts, squares.mV)
show(fig)

In [ ]:
approx_amplitude = squares.mV.max() - squares.mV.min()
fourier_coefs = np.fft.fft(squares.mV, n=len(squares)*8)
fourier_freqs = np.fft.fftfreq(
    len(squares.mV)*8, squares_config['sample_interval'][0]
)

pulse_frequency = fourier_freqs[
    10:len(fourier_freqs)//2][
    fourier_coefs[10:len(fourier_freqs)//2].argmax()
]

approx_amplitude, 'mV', pulse_frequency, 'Hz'

In [ ]:
fig = figure()
fig.line(fourier_freqs, np.abs(fourier_coefs))
show(fig)

## Initial Optics Setup

#### Failed Data

In [ ]:
[[pd1, pd1_config], [pd2, pd2_config]] = read_scope_csv(
    './Data/l4_C_data.csv', 2)

fig = figure()
fig.line(pd1.ts, pd1.mV, legend_label='PD1')
fig.line(pd1.ts, pd2.mV, legend_label='PD2', color='red')
show(fig)

In [ ]:
[[pd1, pd1_config], [pd2, pd2_config]] = read_scope_csv(
    './Data/l4_C_data3.csv', 2)

fig = figure()
fig.line(pd1.ts, pd1.mV, legend_label='PD1')
fig.line(pd1.ts, nu.fft_filter(pd2.mV, .17), legend_label='PD2', color='red')
show(fig)

### Loading Data

In [ ]:
[[pd1, pd1_config], [pd2, pd2_config]] = read_scope_csv(
    './Data/l4_C_data4.csv', 2)

fig = figure()
fig.line(pd1.ts, pd1.mV, legend_label='PD1')
fig.line(pd1.ts, pd2.mV, legend_label='PD2', color='red')
show(fig)

In [ ]:
[[pd1_cable, pd1_config], [pd2_cable, pd2_config]] = read_scope_csv(
    './Data/l4_C_c_data.csv', 2)

fig = figure()
fig.line(pd1_cable.ts, pd1_cable.mV, legend_label='PD1_cable')
fig.line(pd1_cable.ts, pd2_cable.mV, legend_label='PD2_cable', color='red')
show(fig)

### Rising Edge analysis

In [ ]:
from bokeh.palettes import Category20_4 as colors

In [ ]:
from Helper.numbers import rising_edge

#### First pass

In [ ]:
fig = figure()

"""
Create a time vector beginning at zero for alignment purposes.
"""
t = pd1.loc[pd1.ts >= 0, 'ts']
t.index = range(len(t))

"""
Filter all curves such that they are identical. This is really for
ease of processing but also because they really should be identical,
and any differences must represent some error.
"""
pd1_cable_clean = pd.Series(nu.fft_filter(pd1_cable.mV, .44))
pd2_cable_clean = pd.Series(nu.fft_filter(pd2_cable.mV, .34))

pd1_clean = pd.Series(nu.fft_filter(pd1.mV, .34))
pd2_clean = pd.Series(nu.fft_filter(pd2.mV, .34))

"""
Caclulate the indeces of the rising edge for all signals, and the
locations of 10% and 90%.
"""
pd1_cable_inds, pd1_cable_90_10_inds = rising_edge(
    pd1_cable_clean, pd1_cable.ts)
pd1_inds, pd1_90_10_inds = rising_edge(
    pd1_clean, pd1_cable.ts)

pd2_cable_inds, pd2_cable_90_20_inds = rising_edge(
    pd2_cable_clean, pd2_cable.ts)
pd2_inds, pd2_90_20_inds = rising_edge(
    pd2_clean, pd2_cable.ts)

"""
Align each PD1 curve to the beginning of its rising edge.
Align each PD2 curve to the beginning of the rising edge of
the corresponding PD1.
"""
zi_pd1_cable = pd1_cable_clean[pd1_cable_inds[0]:]
zi_pd1 = pd1_clean[pd1_inds[0]:]

zi_pd2_cable = pd2_cable_clean[pd1_cable_inds[0]:]
zi_pd2 = pd2_clean[pd1_inds[0]:]

"""
Normalize each signal on the scale [0, 1].
"""
zi_pd1_cable -= zi_pd1_cable.min()
zi_pd2_cable -= zi_pd2_cable.min()
zi_pd1_cable /= zi_pd1_cable.max()
zi_pd2_cable /= zi_pd2_cable.max()

zi_pd1 -= zi_pd1.min()
zi_pd2 -= zi_pd2.min()
zi_pd1 /= zi_pd1.max()
zi_pd2 /= zi_pd2.max()

"""
Plot each pair of PD1 and PD2 signals, and calculate
their locations of etc.
"""
results = {}
for sig, name, color in zip(
    [zi_pd1_cable, zi_pd1, zi_pd2_cable, zi_pd2],
    ['PD1_cable', 'PD1', 'PD2_cable', 'PD2'],
    colors
):
    sig.index = range(len(sig))
    fig.line(t,
             sig[:len(t)],
             legend_label=name,
             color=color,
             line_width=1.6)
    times = rising_edge(sig[:len(t)], t)
    times = times[0] + times[1]
    results.update({name:[t[i] for i in times]})
    
fig.x_range = Range1d(-.4e-9, 3e-8)
show(fig)
results = pd.DataFrame(results, index = ['start','stop','10','90']).T

In [ ]:
diffs = pd.DataFrame(results.T[['PD2_cable', 'PD2']].values -
                     results.T[['PD1_cable', 'PD1']].values,
                     index=results.columns,
                     columns=['cable_diff', 'no-cable_diff'])
(diffs.cable_diff-diffs['no-cable_diff']).mean(), (diffs.cable_diff-diffs['no-cable_diff']).std()

#### Second Pass

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
res = clean_vis_9010_diff(pd1, pd2)
res.iloc[1] - res.iloc[0]

In [ ]:
res2 = clean_vis_9010_diff(pd1_cable, pd2_cable)
res1 = clean_vis_9010_diff(pd1, pd2)
(res2.iloc[1] - res2.iloc[0])-(res1.iloc[1] - res1.iloc[0])

In [ ]:
difdif = ((res2.iloc[1] - res2.iloc[0])-(res1.iloc[1] - res1.iloc[0]))
'Added time due to the longer cable mean and standard deviation: ', difdif.mean(), difdif.std()

In [ ]:
944.5e-3/difdif.mean()

In [ ]:
2.9979e8/191673016

## D

### Air

#### Setup Distances

In [ ]:
air_pd2_path = ufloat(917.5, .5) + ufloat(730.0, .5) + ufloat(124.5, .5)
air_pd1_path = ufloat(168.5, .5)
air_length_diff = air_pd2_path - air_pd1_path
air_length_diff *= 1e-3 # convert to meters
air_length_diff

#### Loading Data

In [ ]:
[[pd1_air, air_config], [pd2_air, air_config]] = read_scope_csv(
    './Data/l4_D_1_b.csv', 2)
pd1_air.loc[pd1_air.ts > 6e-9, 'mV'] = 0
pd2_air.loc[pd2_air.ts > 1.1e-8, 'mV'] = 0

fig = figure()

fig.line(pd1_air.ts, pd1_air.mV, legend_label='pd1_air')
fig.line(pd1_air.ts, pd2_air.mV, legend_label='pd2_air', color='red')
show(fig)

#### Time Shift

In [ ]:
error_opt = pd.DataFrame(columns=['param', 'mean', 'std'])
error_opt.param = np.arange(.05, .55, .02)


def fun(row):
    filter_param = row.param
    air_firstpeak_times = clean_vis_9010_diff(
        pd1_air, pd2_air, False, filter_param=filter_param, plot=False)
    firstpeak_air_vals = (
        air_firstpeak_times.iloc[1] - air_firstpeak_times.iloc[0])
    row[['mean', 'std']] = firstpeak_air_vals.mean(), firstpeak_air_vals.std()
    return row


error_opt = error_opt.apply(fun, axis=1)

error_opt

In [ ]:
firstpeak_air_vals = (error_opt.iloc[1] - error_opt.iloc[0])
air_firstpeak_timeshift = ufloat(firstpeak_air_vals.mean(), firstpeak_air_vals.std())
nu.print_unc(air_firstpeak_timeshift)

In [ ]:
air_firstpeak_times = clean_vis_9010_diff(
    pd1_air, pd2_air, False, filter_param=.51, plot=True)
air_firstpeak_times

In [ ]:
air_timeshift = error_opt.iloc[22, 1:]
air_timeshift = ufloat(air_timeshift['mean'], air_timeshift['std'])
air_timeshift

#### Refractive Index

In [ ]:
nu.print_unc(299792458/(air_length_diff/air_timeshift))

### Glass

#### Cable Length

In [ ]:
FO_legth = ufloat(2065.5, .5)*1e-3

#### Loading Data

In [ ]:
[[pd1_glass, glass_config], [pd2_glass, glass_config]] = read_scope_csv(
    './Data/l4_D_2_b.csv', 2)

fig = figure()
pd1_glass.loc[pd1_glass.ts > 6e-9, 'mV'] = 0
pd2_glass.loc[pd2_glass.ts > 2.1e-8, 'mV'] = 0
fig.line(pd1_glass.ts, pd1_glass.mV, legend_label='pd1_glass')
fig.line(pd1_glass.ts, pd2_glass.mV, legend_label='pd2_glass', color='red')
show(fig)

#### Timeshift

In [ ]:
glass_peak_times = clean_vis_9010_diff(pd1_glass, pd2_glass, False, 'fft', .2)

glass_timeshift = (glass_peak_times.iloc[1] - glass_peak_times.iloc[0])
glass_timeshift = ufloat(glass_timeshift.mean(), glass_timeshift.std())
glass_timeshift

#### Refractive Index

In [ ]:
FO_shift = glass_timeshift - air_timeshift

In [ ]:
nu.print_unc(2.998e8/(FO_legth / FO_shift))

### Water

#### Water length

In [ ]:
water_length = ufloat(614.0, .5)*1e-3

#### Loading Data

In [ ]:
[[pd1_water, water_config], [pd2_water, water_config]] = read_scope_csv(
    './Data/l4_D_3_b.csv', 2)

pd1_water.loc[pd1_water.ts > 6e-9, 'mV'] = 0
pd2_water.loc[pd2_water.ts > 1.25e-8, 'mV'] = 0

fig = figure()
fig.line(pd1_water.ts, pd1_water.mV, legend_label='pd1_water')
fig.line(pd1_water.ts, pd2_water.mV, legend_label='pd2_water', color='red')
show(fig)

#### Time Shift

In [ ]:
water_peak_times = clean_vis_9010_diff(pd1_water, pd2_water, False, 'fft', 1.6)

water_air_timeshift = (water_peak_times.iloc[1] - water_peak_times.iloc[0])
water_air_timeshift = ufloat(water_air_timeshift.mean(), water_air_timeshift.std())
water_air_timeshift

#### Refractive Index

$$T_A = time\ of\ flight\ diff\ air$$
$$T_W = time\ of\ flight\ in\ water$$
$$D_A = distance\ of\ air\ flight\ path$$
$$D_W = distance\ of\ water\ flight\ path$$
$$T_{A-W} = T_A \cdot \frac{D_A-D_W}{D_A}$$
$$T_W = T_A - T_{A-W}$$

In [ ]:
time_not_water = air_timeshift * (air_length_diff-2*water_length)/air_length_diff
water_timeshift = water_air_timeshift - time_not_water

water_speed = 2*water_length/water_timeshift
(2.998e8/water_speed)